# Preprocess Target Data





In [0]:
from google.colab import drive
import os
drive.mount('/content/drive')
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


<h2>Apply labels to Target dataframe</h2>

In [0]:
import os
import pandas as pd
from PIL import Image 
from IPython.display import Image as IM
from time import sleep


#create df
columns = ['Scan','Label']
df = pd.DataFrame(columns=columns)


os.chdir('drive/My Drive')
encoding = {'CT_COVID': 1, 'CT_NonCOVID':0}

#check encoding of directory then add to df
index = 0
for (dirs, root, files) in os.walk(os.getcwd()):
  rel_dir = os.path.relpath(dirs,os.getcwd())
  if rel_dir in encoding:
    dir_path = os.path.join(os.getcwd(),rel_dir)
    for scan in files:
      df.loc[index] = os.path.join(dir_path,scan), encoding[rel_dir]
      index += 1



CT_NonCOVID
397
CT_COVID
349
                                           Scan Label
0  /content/drive/My Drive/CT_NonCOVID/1702.png     0
1  /content/drive/My Drive/CT_NonCOVID/1310.png     0
2   /content/drive/My Drive/CT_NonCOVID/782.png     0
3   /content/drive/My Drive/CT_NonCOVID/663.png     0
4   /content/drive/My Drive/CT_NonCOVID/781.png     0


<h2>Navigate to Covid Sets</h2>

In [0]:
import os
print(os.getcwd())
os.chdir('drive/My Drive/covid_sets')
print(os.getcwd())

/content
/content/drive/My Drive/covid_sets


<h2>Read Target Metadata</h3>

In [0]:
import pandas as pd

df = pd.read_csv(os.path.join(os.getcwd(),'target_metadata.csv'))

<h2>Load Images</h2>

In [0]:
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import cv2
from time import sleep
from PIL import Image
from keras.utils import to_categorical
import math

data = []
labels = []

for index,row in df.iterrows():
  if index % (746//10) == 0 and index!= 0:
    print(f'{math.ceil((index/746)*100)}% complete')
  scan = df.loc[index,'Scan']
 
  img = cv2.imread(scan)
  processed_img = cv2.resize(img,(224,224),3)

  data.append(processed_img)
  labels.append(df.loc[index,'Label'])
  





Using TensorFlow backend.


10% complete
20% complete
30% complete
40% complete
50% complete
60% complete
70% complete
80% complete
90% complete
100% complete
746 746


<h2>Split Data</h2>

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np

#split into 75:25 train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25)
X_test, y_test = np.array(X_test), np.array(y_test)

#split train set into 80:20 train and val sets
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.20)
X_train, y_train, X_val, y_val, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_val), np.array(y_val), np.array(X_test), np.array(y_test)

#store split datasets
datasets = {'X_train': X_train,'y_train':y_train,'X_val':X_val,'y_val':y_val,'X_test':X_test,'y_test':y_test}

covid samples in training set: 252, covid samples in test set: 97
normal samples in training set: 307, normal samples in test set: 90


<h2>Save Split Data</h2>

In [0]:
import numpy as np

for key,value in datasets.items():
  np.save(os.path.join(os.getcwd(),key),value)
